# Sayou Loader Quick Start

**Sayou Fabric**의 최종착륙지인 `sayou-loader`를 시연합니다.
이 라이브러리는 조립된 데이터(Assembler Output)를 실제 저장소에 **안전하게(Retry)** 적재합니다.

**시연 시나리오:**
1. **File Loading:** JSON 포맷으로 로컬 저장
2. **Stream Loading:** 대량 데이터를 JSONL(Line-delimited JSON)로 저장
3. **Mock DB Loading:** Neo4j 적재 시뮬레이션

In [ ]:
import os
import shutil
import logging
from sayou.loader.pipeline import LoaderPipeline

# 로그 설정
logging.basicConfig(level=logging.INFO, format='%(message)s')

print("Import Successful!")

## 1. 파이프라인 초기화

Loader 파이프라인을 생성합니다. 별도의 설정 없이도 기본 파일/콘솔 라이터가 로드됩니다.

In [ ]:
pipeline = LoaderPipeline()
pipeline.initialize()

# 테스트용 출력 디렉토리 정리
OUTPUT_DIR = "examples"
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR)

## 2. 시나리오 1: 그래프 데이터 저장 (JSON)

Assembler가 만든 그래프 구조(Dict)를 JSON 파일로 저장합니다.
`FileWriter`는 `destination`이 폴더면 자동으로 파일명을 생성하고, 데이터가 Dict면 JSON으로 직렬화합니다.

In [ ]:
# 더미 그래프 데이터
graph_payload = {
    "nodes": [
        {"id": "n1", "label": "Person", "props": {"name": "Alice"}},
        {"id": "n2", "label": "Person", "props": {"name": "Bob"}}
    ],
    "edges": [
        {"source": "n1", "target": "n2", "type": "KNOWS"}
    ],
    "metadata": {"generated_at": "2023-10-27"}
}

# 전략: file
dest_path = os.path.join(OUTPUT_DIR, "knowledge_graph.json")

pipeline.run(graph_payload, destination=dest_path, strategy="file")

print(f"✅ Check file: {dest_path}")

## 3. 시나리오 2: 대용량 벡터 데이터 저장 (JSONL)

수만 개의 벡터 데이터는 단일 JSON보다 `JSONL` (줄바꿈으로 구분) 형식이 유리합니다.
`JsonLineWriter`를 사용합니다.

In [ ]:
# 더미 벡터 데이터 (리스트)
vector_payloads = [
    {"id": f"vec_{i}", "vector": [0.1, 0.2, float(i)], "text": f"Chunk {i}"}
    for i in range(5)
]

# 전략: jsonl (또는 stream)
dest_path_l = os.path.join(OUTPUT_DIR, "vectors.jsonl")

pipeline.run(vector_payloads, destination=dest_path_l, strategy="jsonl")

print(f"✅ Check file: {dest_path_l}")

# 내용 확인
with open(dest_path_l, "r") as f:
    print("\n[File Content Preview]")
    print(f.read())

## 4. 시나리오 3: DB 적재 시뮬레이션 (Console/Mock)

실제 DB가 없을 때 디버깅용으로 사용하는 `console` 전략과, Mock으로 구현된 `neo4j` 전략을 실행합니다.

In [ ]:
# 콘솔 출력 테스트
pipeline.run("System ready.", destination="STDOUT", strategy="console")

In [ ]:
# Neo4j Mock 테스트 (쿼리 리스트)
cypher_queries = [
    "MERGE (n:Person {name: 'Alice'})",
    "MERGE (n:Person {name: 'Bob'})"
]

# 실제 접속 정보가 없으므로 로그만 출력됨
pipeline.run(
    cypher_queries, 
    destination="bolt://localhost:7687", 
    strategy="neo4j",
    auth=("neo4j", "password") # kwargs로 인증 정보 전달
)